In [ ]:
import pandas as pd
import vectorbt as vbt
import numpy as np

In [ ]:
# Load individual asset close prices
btc = pd.read_csv("BTCUSDT_1h_2025-06-09.csv", parse_dates=True, index_col=0)['close']
eth = pd.read_csv("ETHUSDT_1h_2025-06-09.csv", parse_dates=True, index_col=0)['close']
xrp = pd.read_csv('XRPUSDT_1h_2025-06-09.csv', parse_dates=True, index_col=0)['close']

In [ ]:
# Function to clean and prepare each CSV
def load_and_clean_csv(path, name):
    df = pd.read_csv(path, parse_dates=['timestamp'])
    df = df[['timestamp', 'close']].dropna()
    df = df.groupby('timestamp').mean()  # Handle duplicates by averaging
    df.rename(columns={'close': name}, inplace=True)
    return df

In [ ]:
# Load and clean individual assets
btc = load_and_clean_csv('BTCUSDT_1h_2025-06-09.csv', 'BTC')
eth = load_and_clean_csv('ETHUSDT_1h_2025-06-09.csv', 'ETH')
xrp = load_and_clean_csv('XRPUSDT_1h_2025-06-09.csv', 'XRP')

In [ ]:
# Merge on Date
close = btc.join([eth, xrp], how='outer')
close = close.sort_index()
close = close.dropna()
close

In [ ]:
# Strategy Parameters
sma_period = 50
ema_fast = 9
ema_slow = 21

In [ ]:
# Indicators
sma = close.rolling(window=sma_period).mean()
ema_fast_val = close.ewm(span=ema_fast, adjust=False).mean()
ema_slow_val = close.ewm(span=ema_slow, adjust=False).mean()

In [ ]:
# Entry and Exit Conditions
ema_cross_up = ema_fast_val > ema_slow_val
ema_cross_down = ema_fast_val < ema_slow_val
trend_bull = close > sma
trend_bear = close < sma
price_above_buffer = close > ema_slow_val * 1.005

# Risk management conditions
max_drawdown = 0.20  # 20% drawdown limit
highest_since_entry = highest(close, since(entries))  # tracks highest price since entry
drawdown_breach = close < highest_since_entry * (1 - max_drawdown)

entries = ema_cross_up & trend_bull & price_above_buffer
exits = ema_cross_down & trend_bear | drawdown_breach  # exits when either condition is met

#entries = ema_cross_up & trend_bull & price_above_buffer
#exits = ema_cross_down & trend_bear

In [ ]:
# Backtest Parameters
initial_cash = 100_000
num_assets = close.shape[1]
capital_per_asset = initial_cash / num_assets

In [ ]:
# Run Backtest with no compounding and no cooldown
pf = vbt.Portfolio.from_signals(
    close=close,
    entries=entries,
    exits=exits,
    size=capital_per_asset,
    size_type='amount',         # Use fixed amount per trade
    init_cash=initial_cash,
    fees=0.001,
    cash_sharing=True,          # All assets share the same cash pool
    freq='1h'                   # Set frequency to avoid ratio warnings
)

In [ ]:
# Output results
print(pf.stats())
pf.plot().show()

In [ ]:
pf.plot_drawdowns(title='Drawdown Over Time').show()


In [ ]:
pf.returns()


In [ ]:
#pf.returns.cumsum().vbt.plot(title='Cumulative Returns per Asset').show()
pf.returns().cumsum().vbt.plot(title='Cumulative Returns per Asset').show()


In [ ]:
#Using the compound capital, ie, all the available capital for entering each trade

In [ ]:
pf1 = vbt.Portfolio.from_signals(
    close=close,
    entries=entries,
    exits=exits,
    size=1.0,                   # Use 100% of available capital for each trade
    size_type='percent',        # Use percent of current equity (compounding)
    init_cash=initial_cash,
    fees=0.001,
    cash_sharing=True,
    freq='1h'
)

In [ ]:
pf1.returns()

In [ ]:
pf1.returns().cumsum().vbt.plot(title='Cumulative Returns per Asset').show()

In [ ]:
# Output results
print(pf1.stats())
pf1.plot().show()

In [ ]:
pf1.plot_drawdowns(title='Drawdown Over Time').show()

In [ ]:
entries_bh = close.index == close.index[0]  # True only on the first timestamp
exits_bh = pd.Series(False, index=close.index)  # Never exit

pf_bh = vbt.Portfolio.from_signals(
    close=close,
    entries=entries_bh,
    exits=exits_bh,
    init_cash=pf.init_cash,
    fees=0.0,              # Usually buy and hold assumes zero trading fees
    freq=pf.freq
)


In [ ]:
entries_bh = close.index == close.index[0]  # True only on the first timestamp
exits_bh = pd.Series(False, index=close.index)  # Never exit

pf_bh = vbt.Portfolio.from_signals(
    close=close,
    entries=entries_bh,
    exits=exits_bh,
    init_cash=pf.init_cash,  # use your initial cash
    fees=0.0,
    freq=close.index.freq or close.index.inferred_freq  # Get freq from index
)


In [ ]:
#only SMA and EMA strategy

In [ ]:
# Entry and Exit Conditions (simplified)
ema_cross_up = ema_fast_val > ema_slow_val
ema_cross_down = ema_fast_val < ema_slow_val
trend_bull = close > sma
trend_bear = close < sma

# Final signals
entries = ema_cross_up & trend_bull
exits = ema_cross_down & trend_bear


In [ ]:
pf2 = vbt.Portfolio.from_signals(
    close=close,
    entries=entries,
    exits=exits,
    size=1.0,                   # 100% of current equity (compounding)
    size_type='percent',
    init_cash=initial_cash,
    fees=0.002,                 # Simulate 0.2% slippage
    cash_sharing=True,
    freq='1h'
)


In [ ]:
# Output results
print(pf2.stats())
pf2.plot().show()